## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
convention_db = sqlite3.connect("/Users/JohnnyBlaze/2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [3]:
# This code retrieves a list of tables from the SQLite databases

# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master
WHERE type='table';"""
    
# Executing our SQL query
convention_cur.execute(sql_query)
    
print("The list of tables names\n")
    
# Printing all tables list
print(convention_cur.fetchall())

The list of tables names

[('conventions',)]


In [4]:
# Set up stopwords and punctuation for text processing
stop_words = set(stopwords.words('english'))
punctuations = set(punctuation)

# Function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    final_tokens = []
    for word in tokens:
        if word not in stop_words:
            final_tokens.append(word)
    return final_tokens

# Function to tokenize text into a list of words
def tokenize(text):
    return text.split()

# Function to remove punctuation from text
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

# Define a text preparation pipeline
pipeline = [str.lower, remove_punctuation, tokenize, remove_stopwords]

# Function to prepare text using a given pipeline of transformations
def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

In [5]:
convention_data = []
# Fill this list up with items that are themselves lists.
# The first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party.
query_results = convention_cur.execute(
    '''
    SELECT party, text from conventions;
    '''
)

convention_data = [[token_string, row[0]] for row in query_results for token_string in [" ".join(prepare(row[1], pipeline))]]

Let's look at some random entries and see if they look right. 

In [6]:
random.choices(convention_data,k=10)

[['put opportunity zones trump tax bill would drive investment communities decades come put interest american workers especially black workers first that’s right donald trump delivered historic criminal justice reform ended policy incarceration black people decimated communities caused joe biden democrats wouldn’t obama didn’t want joe biden kamala harris definitely wouldn’t donald trump he’s also working every day make community safer former executive dekalb county georgia directed one largest public safety departments southeast',
  'Republican'],
 ['justice strength liberty', 'Democratic'],
 ['daughter murdered parkland joe biden called share family’s grief quickly learned decency civility also learned toughness he’s beaten nra together victims gun violence nation’s youth joe biden kamala harris take nra win let’s win back freedom live without fear florida cast 57 votes bernie sanders 192 votes next president joe biden',
  'Democratic'],
 ['important thing say survivor hear that’s be

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [7]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [8]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = {t: True for t in text.split() if t in fw}

    return ret_dict

In [9]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [10]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [11]:
random.seed(20230604)
random.shuffle(featuresets)

test_size = 500

In [12]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.528


In [13]:
classifier.show_most_informative_features(25)

Most Informative Features
                   media = True           Republ : Democr =     33.0 : 1.0
             enforcement = True           Republ : Democr =     30.9 : 1.0
                   votes = True           Democr : Republ =     24.4 : 1.0
                   china = True           Republ : Democr =     19.1 : 1.0
                   taxes = True           Republ : Democr =     16.7 : 1.0
                 destroy = True           Republ : Democr =     16.3 : 1.0
                freedoms = True           Republ : Democr =     15.2 : 1.0
                    cuts = True           Republ : Democr =     14.2 : 1.0
                    iran = True           Republ : Democr =     13.1 : 1.0
                supports = True           Republ : Democr =     13.1 : 1.0
                    kept = True           Republ : Democr =     12.9 : 1.0
                   trade = True           Republ : Democr =     12.8 : 1.0
                    mike = True           Republ : Democr =     12.3 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting? 

### My Observations

When I look at these results it appears that Republicans have the most charged words. The top two words used are media and enforcement. In this list the Democrats only have one charged word which is votes. We can see this list is dominated by Republican words and get an good idea of what words we see in their politics.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [14]:
cong_db = sqlite3.connect("/Users/JohnnyBlaze/congressional_data.db")
cong_cur = cong_db.cursor()

In [15]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [16]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

tweet_data = [[token_str, tweet[1]] for tweet in results for token_str in [" ".join(prepare(tweet[2].decode(), pipeline))]]

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [17]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

print(tweet_data_sample)

[['earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv', 'Democratic'], ['go tribe rallytogether httpstco0nxutfl9l5', 'Democratic'], ['apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh', 'Democratic'], ['we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3', 'Republican'], ['let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z', 'Republican'], ['1hr cavs tie series 22 im allin216 repbarbaralee scared roadtovictory', 'Democratic'], ['congrats belliottsd new gig sd city hall glad continue serve… httpstcofkvmw3cqdi', 'Democratic'], ['really close 3500 raised toward match right whoot that’s 7000 nonmath majors room 😂 help us get httpstcotu34c472sd httpstcoqsdqkypsmc', 'Democratic'], ['today comment period potus’s plan expand offshor

In [18]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [19]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [20]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3589, 'Democratic': 689}),
             'Democratic': defaultdict(int,
                         {'Republican': 4657, 'Democratic': 1067})})

### Reflections

_Write a little about what you see in the results_ 

This model is better at classifiying Republicans. It appears that the word list was imbalanced and heavily in favor of Republican, which might be the reason why it does a better job with classifying Republicans.